這裏的程式碼其實也就是 Deeplearning.ai "Building Systems with the ChatGPT API" 課程裏面的程式碼的轉譯版。
大家可以參考原始課程： [Building Systems with the ChatGPT API](https://learn.deeplearning.ai/chatgpt-building-system/)

## 環境的準備： ##
1. 使用 os.environ 讀取 openai api key, 請把你的 key 寫到本專案的 .env 檔案內。
2. get_completion_from_messages 是工具函式，後方很多地方都會用到它來取得回應。

In [2]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [3]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

## 將使用者的訊息做分類 ##

In [4]:
delimiter = "####"
system_message = f"""
將會收到一些客戶服務查詢。 \
每個客戶服務查詢將以 {delimiter} 字元進行分隔。 \
請將每個查詢分類為主要類別和次要類別。 \
並以 json 格式提供你的輸出，鍵值為：primary（主類別）和 secondary（次類別）。 \

Primary（主類別）: 帳單、技術支援、帳戶管理或一般查詢

帳單的次類別:
取消訂閱或升級
增加付款方式
收費說明
爭議收費

技術支援次類別:
一般故障排除
設備相容性
軟體更新

帳戶管理次要類別:
重設密碼
更新個人資訊
關閉帳戶
帳戶安全

一般查詢的次要類別:
產品資訊
價格
反饋
與真人客服對話

"""
user_message = f"""\
我要你刪除我的個人資料和所有使用者資料"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

{
    "primary": "帳戶管理",
    "secondary": "關閉帳戶"
}
